## Deep riccy project

## Colab dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! tar -zxvf /content/drive/MyDrive/Uni/DeepRiccy/refcocog.tar.gz

In [1]:
#set local path to the data

#this is for colab
#local_path = '/content/refcocog/images/'
#local_annotations = '/content/refcocog/annotations/'

#this is for local
local_path = './refcocog/images/' 
local_annotations = './refcocog/annotations/' 

## Dataset

In [2]:
#import section
import json
import pickle
import torch
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
import pandas as pd


### pandas

In [ ]:
class Coco(Dataset):
    def __init__(self, path_json, path_pickle, train=True):
        self.path = path_json
        self.path_pickle = path_pickle
        self.train = train

        #load images
        with open(self.path) as json_data:
            data = json.load(json_data)
            self.img_frame = pd.DataFrame(data['images'])
            self.img_frame = self.img_frame.set_index('id')

        #load annotations
        with open(self.path) as json_data:
            data = json.load(json_data)
            self.ann_frame = pd.DataFrame(data['annotations'])
            self.ann_frame = self.ann_frame.set_index('id')

        #load annotations
        with open(self.path_pickle, 'rb') as pickle_data:
            data = pickle.load(pickle_data)
            self.refs_frame = pd.DataFrame(data)

        self.size = self.refs_frame.shape[0]
    
    def __len__(self):
        return self.size
    
    def __getitem__(self, idx):
        #get annotation
        annotation = self.refs_frame.loc[idx]

        #get image
        img_id = annotation['image_id']
        img_name = self.img_frame.iloc[img_id]['file_name']
        img = Image.open(local_path+img_name)
        
        #get sentence
        sentence = annotation['sentences'][0]

        #ground truth
        region_id = annotation['region_id']
        gt = self.img_frame.iloc[img_id]['regions'][region_id]




In [77]:
#load dataset with pandas
with open(local_annotations+'instances.json') as json_data:
    data = json.load(json_data)
    img_frame = pd.DataFrame(data['images'])
    img_frame = img_frame.set_index('id')

#load annotations with pandas and pickle
with open(local_annotations+"refs(umd).p", 'rb') as pickle_data:
    data = pickle.load(pickle_data)
    refs_frame = pd.DataFrame(data)

with open(local_annotations+'instances.json') as json_data:
            data = json.load(json_data)
            ann_frame = pd.DataFrame(data['annotations'])
            ann_frame = ann_frame.set_index('image_id')


In [82]:

img_id = refs_frame.iloc[0]['image_id']
sentence = refs_frame.loc[0]['sentences'][0]


annotation = ann_frame.loc[img_id]
bbox = annotation['bbox']

img_name = img_frame.loc[img_id]['file_name']

print(img_name)
print(bbox)
print(sentence)

COCO_train2014_000000380440.jpg
image_id
380440     [228.98, 41.4, 141.95, 270.38]
380440    [374.31, 65.06, 136.04, 201.94]
380440     [244.21, 251.3, 141.07, 90.63]
380440    [291.38, 190.07, 185.19, 68.27]
380440      [416.16, 95.57, 66.98, 58.28]
380440       [288.6, 75.67, 54.76, 27.93]
Name: bbox, dtype: object
{'tokens': ['the', 'man', 'in', 'yellow', 'coat'], 'raw': 'the man in yellow coat', 'sent_id': 8, 'sent': 'the man in yellow coat'}


### json standard

In [ ]:
#class creation for the items
class CocogImage():
    def __init__(self, id, json_file, referece, refg):
        self.refg = refg
        self.id = id
        self.data = json_file['images'][id]
        self.bboxes = []
        self.img_id = self.data['id']

        for i in json_file['annotations']:
            if i['image_id'] == self.img_id:
                self.bboxes.append(i["bbox"])
                
        self.annotations = []

        for i in referece:
            if i['image_id'] == self.img_id:
                self.annotations.append(i["sentences"])
        
        self.refg = []
        for i in refg:
            if i['image_id'] == self.img_id:
                self.refg.append(i)
                    
    def show(self):
        img = Image.open(local_path + self.data['file_name'])
        plt.imshow(img)
        plt.show()

    def show_bbox(self):
        #print image with bbox
        img = self.get_image()
        plt.imshow(img)
        for i in self.bboxes:
            plt.gca().add_patch(plt.Rectangle((i[0], i[1]), i[2], i[3], fill=False, edgecolor='red', linewidth=2))
        plt.show()

    def get_image_tensor(self, transform = transforms.Compose([transforms.PILToTensor()])):
        img = Image.open(local_path + self.data['file_name'])
        return transform(img)
    
    def get_image(self):
        return Image.open(local_path + self.data['file_name'])

    def file_name(self):
        return local_path+self.data['file_name']

    def id(self):
        return self.data['id']

    def width(self):
        return self.data['width']

    def height(self):
        return self.data['height']

    def bbox(self):
        return self.bboxes
    
    def annotations(self):
        return self.annotations

In [ ]:
class Coco(Dataset):
  def __init__(self, size):
    super(Coco, self).__init__()
    self.size = size
    #load json file
    with open(local_annotations + 'instances.json', 'r') as f:
      self.data = json.load(f)
    #load references(umd)
    with open(local_annotations + 'refs(umd).p', 'rb') as f:
      self.references = pickle.load(f)
      f.close()
    #load references(google)
    with open(local_annotations + 'refs(google).p', 'rb') as f:
      self.references_google = pickle.load(f)
      f.close()
    
    
  def __getitem__(self, idx):
    if idx >= self.size:
      raise IndexError()
    #import image form json
    image = CocogImage(idx, self.data, self.references, self.references_google)
    return image
    
    return image.get_image(), image.bbox(), image.annotations()

  def __len__(self):
    return self.size


## Model

In [ ]:
#test
dataset = Coco(200)

In [ ]:
#print the first 10 images side by side
fig, axs = plt.subplots(2, 5)
fig.set_size_inches(18.5, 10.5)
for i in range(10):
    axs[i//5, i%5].imshow(dataset[i].get_image())
    axs[i//5, i%5].set_title(str(dataset[i].id))
plt.show()


In [ ]:
#show image 1 with bbox
dataset[1].show()
print("cacca")

### Test dataset and show bboxes

In [ ]:
#get how many annotation have the img_id equal to 131074
count = 0

for i in dataset[0].refg:
  if i['image_id'] == 131074:
    count += 1
print(count)

In [ ]:
with open(local_annotations + 'refs(umd).p', 'rb') as f:
      references = pickle.load(f)

print(references[0])

In [ ]:
#find in the pickle file the annotation with img_id 131074
for i in dataset[0].refg:
  if i['image_id'] == 131074:
    print(i)

### yolo

In [ ]:
#install dependencies
! pip3 install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt 


In [ ]:
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
# Images
imgs = dataset[1].file_name() 
# Inference
results = model(imgs)

# Results
results.print()
results.show()  # or .show()

results.xyxy[0]  # img1 predictions (tensor)
results.pandas().xyxy[0]  # img1 predictions (pandas)

In [ ]:
import numpy as np
#save an image for each bounding box created by yolov5
for i in range(len(results.xyxy[0])):
    x1 = results.xyxy[0][i][0].numpy()
    x1 = np.rint(x1)
    y1 = results.xyxy[0][i][1].numpy()
    y1 = np.rint(y1)
    x2 = results.xyxy[0][i][2].numpy()
    x2 = np.rint(x2)
    y2 = results.xyxy[0][i][3].numpy()
    y2 = np.rint(y2)
    print(x1, y1, x2, y2)
    img = Image.open(dataset[1].file_name())
    img = img.crop((x1, y1, x2, y2))
    img.save("img"+str(i)+".jpg")


## Clip implementation

In [ ]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

In [ ]:
import numpy as np
import torch
from pkg_resources import packaging

print("Torch version:", torch.__version__)

In [ ]:
import clip

clip.available_models()

In [ ]:
model, preprocess = clip.load("ViT-B/32")
model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

In [ ]:
import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from collections import OrderedDict
import torch

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# images in skimage to use and their textual descriptions
descriptions = {
    "page": "a page of text about segmentation",
    "chelsea": "a facial photo of a tabby cat",
    "astronaut": "a portrait of an astronaut with the American flag",
    "rocket": "a rocket standing on a launchpad",
    "motorcycle_right": "a red motorcycle standing in a garage",
    "camera": "a person looking at a camera on a tripod",
    "horse": "a black-and-white silhouette of a horse", 
    "coffee": "a cup of coffee on a saucer"
}

In [ ]:
original_images = []
images = []
texts = []
plt.figure(figsize=(16, 5))

for filename in [filename for filename in os.listdir(skimage.data_dir) if filename.endswith(".png") or filename.endswith(".jpg")]:
    name = os.path.splitext(filename)[0]
    if name not in descriptions:
        continue

    image = Image.open(os.path.join(skimage.data_dir, filename)).convert("RGB")
  
    plt.subplot(2, 4, len(images) + 1)
    plt.imshow(image)
    plt.title(f"{filename}\n{descriptions[name]}")
    plt.xticks([])
    plt.yticks([])

    original_images.append(image)
    images.append(preprocess(image))
    texts.append(descriptions[name])

plt.tight_layout()

In [ ]:
image_input = torch.tensor(np.stack(images)).cuda()
text_tokens = clip.tokenize(["This is " + desc for desc in texts]).cuda()

In [ ]:
with torch.no_grad():
    image_features = model.encode_image(image_input).float()
    text_features = model.encode_text(text_tokens).float()

In [ ]:
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T

In [ ]:
count = len(descriptions)

plt.figure(figsize=(20, 14))
plt.imshow(similarity, vmin=0.1, vmax=0.3)
# plt.colorbar()
plt.yticks(range(count), texts, fontsize=18)
plt.xticks([])
for i, image in enumerate(original_images):
    plt.imshow(image, extent=(i - 0.5, i + 0.5, -1.6, -0.6), origin="lower")
for x in range(similarity.shape[1]):
    for y in range(similarity.shape[0]):
        plt.text(x, y, f"{similarity[y, x]:.2f}", ha="center", va="center", size=12)

for side in ["left", "top", "right", "bottom"]:
  plt.gca().spines[side].set_visible(False)

plt.xlim([-0.5, count - 0.5])
plt.ylim([count + 0.5, -2])

plt.title("Cosine similarity between text and image features", size=20)